In [0]:
## NOTEBOOK DESENVOLVIDO PARA EXECUTAR TESTES UNITARIOS NO OBJETO CRIADO

In [0]:
# %sql
# drop table if exists dados_fake.asso_conta_movi

In [0]:
## EXECUTANDO TESTES UNITÁRIOS NO OBJETO
testes_unitarios = None
try:
    ## VERIFICA SE A TABELA EXISTE
    teste_status = "TABELA NAO EXISTE"
    df_existe = spark.sql("""
        SELECT COUNT(*) AS tabela_existe
        FROM information_schema.tables
        WHERE table_schema = 'dados_fake' AND table_name = 'asso_conta_movi'
    """)
    if df_existe.collect()[0]["tabela_existe"] != 1:
        raise Exception()
    else:

        ## VERIFICA SE ID_MOVIMENTACAO TEM VALORES NULOS
        teste_status = "ID_MOVIMENTOCAO TEM VALORES NULL"
        df_id_movimentacao_nulos = spark.sql("""
            SELECT COUNT(*) AS id_movimentacao_nulos
            FROM dados_fake.asso_conta_movi
            WHERE id_movimentacao IS NULL
        """)
        if df_id_movimentacao_nulos.collect()[0]["id_movimentacao_nulos"] > 0:
            raise Exception()
        else:

            ## VERIFICA SE OS VALORES SÂO NUMERICOS
            teste_status = "VLR_TRANSACAO TEM VALORES NULL"
            df_vlr_transacao_nao_numerico = spark.sql("""
                SELECT COUNT(*) AS vlr_transacao_nao_numerico
                FROM dados_fake.asso_conta_movi
                WHERE TRY_CAST(vlr_transacao AS DOUBLE) IS NULL AND vlr_transacao IS NOT NULL
            """)
            if df_vlr_transacao_nao_numerico.collect()[0]["vlr_transacao_nao_numerico"] > 0:
                spark.sql("SELECT raise_error('Existem valores não numéricos em vlr_transacao')")
            else:

                ## VERIFICA SE EXISTE VALORES NEGATIVOS
                teste_status = "VLR_TRANSACAO TEM VALORES NEGATIVOS"
                df_vlr_transacao_negativos = spark.sql("""
                    SELECT COUNT(*) AS vlr_transacao_negativos
                    FROM dados_fake.asso_conta_movi
                    WHERE TRY_CAST(vlr_transacao AS DOUBLE) < 0
                """)
                if df_vlr_transacao_negativos.collect()[0]["vlr_transacao_negativos"] > 0:
                    raise Exception()
                else:

                    ## VERIFICA SE A TABELA REGISTRA DATA FUTURA
                    teste_status = "TABELA COM DATA FUTURA"
                    df_transacoes_futuras = spark.sql("""
                        SELECT COUNT(*) AS transacoes_futuras
                        FROM dados_fake.asso_conta_movi
                        WHERE TO_DATE(dt_transacao) > CURRENT_DATE()
                    """)
                    if df_transacoes_futuras.collect()[0]["transacoes_futuras"] > 0:
                        raise Exception()
                    else:
                        
                        ## VERIFICA SE TODOS OS EMAILS TEM @
                        teste_status = "EMAIL SEM @"
                        df_emails_invalidos = spark.sql("""
                            SELECT COUNT(*) AS emails_invalidos
                            FROM dados_fake.asso_conta_movi
                            WHERE email NOT LIKE '%@%'
                        """)
                        if df_emails_invalidos.collect()[0]["emails_invalidos"] > 0:
                            raise Exception()
                        else:

                            ## VERIFICA SE UNIQUE_ID É ÚNICO
                            teste_status = "CONSIDERANDO UNIQUE_ID A TABELA NAO TEM UNICIDADE"
                            df_unique_id_dup = spark.sql("""
                                SELECT COUNT(*) AS dup_count
                                FROM (
                                    SELECT UNIQUE_ID, COUNT(*) AS cnt
                                    FROM dados_fake.asso_conta_movi
                                    GROUP BY UNIQUE_ID
                                    HAVING cnt > 1
                                ) t
                            """)
                            if df_unique_id_dup.collect()[0]["dup_count"] > 0:
                                raise Exception()
                            else:
                                testes_unitarios = "OK"
except Exception as e:
    print(e)
    testes_unitarios = "Error"

In [0]:
## EXPORTANDO DADOS EM FORMATO DELTA E PARQUET
if testes_unitarios == "OK":

    ## LENDO A TABELA
    df_export = spark.table("""dados_fake.asso_conta_movi""")

    ## EXPORT TIPO DELTA
    df_export.write.mode("overwrite").format("delta").save("/Volumes/workspace/default/desafio_tecnico/export_data//asso_conta_movi.delta")

    ## EXPORT TIPO PARQUET
    df_export.write.mode("overwrite").format("parquet").save("/Volumes/workspace/default/desafio_tecnico/export_data//asso_conta_movi.parquet")

    print(f"Objeto exportado em formato DELTA E PARQUET'")
else:
    print(f"O objeto não foi aprovado nos testes unitarios: '{teste_status}'")